In [1]:
from math import floor
import numpy as np
import matplotlib.pyplot as plt
from ase.db import connect
import asr

In [2]:
from asr.database import parse_row_data

In [3]:
db = connect('c2db.db')

# Select a row that has HSE06 data:
bandstructure_results = []
for row in db.select('has_asr_bandstructure'):
    data = parse_row_data(row.data)
    bs = data['results-asr.bandstructure.json']
    bandstructure_results.append(bs)
#record = row.cache.get(name='asr.c2db.hse:main')
#result = record.result
#
#print(result.kvbm)
#print(result.kcbm)
bandstructure_results

[]

In [6]:
db = connect('c2db.db')

# Select a row that has HSE06 data:
for row in db.select(has_asr_hse=True):
    break

record = row.cache.get(name='asr.c2db.hse:main')
result = record.result

print(result.kvbm)
print(result.kcbm)

AttributeError: 'NoneType' object has no attribute 'cache'

In [5]:
# Connect to database
db = connect('c2db.db')

rows = db.select('gap>0,class=TMDC-H', sort='gap')
rows

<generator object Database.select at 0x7f2c0d788740>

In [6]:
labels = []
vbms = []
cbms = []
for row in rows:
    M, X = row.symbols[:2]
    label = M + X + '$_2$'
    labels.append(label)
    vbms.append(row.vbm)
    cbms.append(row.cbm)
cbms

[]

In [ ]:



x = np.arange(len(vbms)) + 0.5
emin = floor(min(vbms)) - 1.0

# Width and height in pixels
ppi = 100
figw = 800
figh = 400

fig = plt.figure(figsize=(figw / ppi, figh / ppi), dpi=ppi)
ax = fig.add_subplot(1, 1, 1)
ax.bar(x, np.array(vbms) - emin, bottom=emin)
ax.bar(x, -np.array(cbms), bottom=cbms)
ax.set_xlim(0, len(labels))
ax.set_ylim(emin, 0)
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=90, fontsize=10)

plt.title("2H-TMD's: Positions of VBM and CBM (PBE+SOC)", fontsize=12)
plt.ylabel('Energy relative to vacuum [eV]', fontsize=10)
plt.tight_layout()
plt.savefig('band-alignment.png')

In [4]:
# creates: fig18.png
import matplotlib.pyplot as plt
from asr.infraredpolarizability import \
    Result as InfraredPolarizabilityResult
from asr.polarizability import Result as PolarizabilityResult
from ase.db import connect
from scipy.interpolate import interp1d

db = connect('c2db.db')
row = db.get(formula='BN')

dct = row.data['results-asr.infraredpolarizability.json']
irpol = InfraredPolarizabilityResult(**dct['kwargs'])
freq = irpol.omega_w
iralpha = irpol.alpha_wvv[:, 0, 0]  # xx-component

dct = row.data['results-asr.polarizability.json']
elpol = PolarizabilityResult(**dct['kwargs'])
x = elpol.frequencies
y = elpol.alphax_w
# Interpolate to IR frequency grid:
elalpha = (interp1d(x, y.real)(freq) +
           interp1d(x, y.imag)(freq) * 1j)

alpha = iralpha + elalpha

ax = plt.subplot()
ax.plot(freq * 1000, alpha.real, label='real')
ax.plot(freq * 1000, alpha.imag, label='imag')
ax.set_xlabel('Energy [meV]')
ax.set_ylabel(r'Polarizability [$\mathrm{\AA}$]')
ax.set_xlim(0, 500)
ax.legend()
plt.tight_layout()
plt.savefig('fig18.png')

KeyError: 'no match'